In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool

from langchain.agents import create_agent
from langchain.messages import HumanMessage

In [ ]:
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
text = [
    'Apple makes great phones.',
    'Apples are red or green in color.',
    'I am a fan of Macbooks.',
    'Bananas are my favorite fruit.',
    'The banana is yellow.'
]

In [ ]:
vector_store = FAISS.from_texts(texts=text, embedding=embeddings)

In [ ]:
print(vector_store.similarity_search('Which fruit is yellow?', k=2))

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={'k': 2})

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='kb_search', #knowledge base search
    description='Search the small knowledge base about fruits / products.'
)

In [ ]:
agent = create_agent(
    tools=[retriever_tool],
    model='gpt-4o',
    System_prompt=(
        'You are a helpful assistant that uses the tool to answer questions about fruits and products, '
        'First use the tool to get relevant information, then answer the question based on the retrieved information.'
    )
)

In [ ]:
result = agent.invoke({
    'messages': [{
            'role': 'user',
            'content': 'What color are bananas and what fruits does the person likes?'
        }]
})

print(result['messages'][-1]['content'])

In [ ]:
# Alternatively, using HumanMessage
result = agent.invoke({
    'messages': [
        HumanMessage(content=['What color are bananas and what fruits does the person likes?'])
    ]
})

print(result['content'])